# Simplified Ikeda regression

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

from rolldecayestimators.simplified_ikeda import calculate_roll_damping
from rolldecayestimators import equations
import sympy as sp
from rolldecayestimators import symbols
from rolldecayestimators.substitute_dynamic_symbols import lambdify
from mdldb.tables import Run
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from sklearn.pipeline import Pipeline

from rolldecay import database

import data
import copy
from rolldecay import database
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

In [ ]:
df_ikeda = database.load('rolldecay_simplified_ikeda',limit_score=0.5,
                         exclude_table_name='rolldecay_exclude')
#mask = (df_ikeda['ship_speed']==0)   ## Zero speed!
#df_ikeda = df_ikeda.loc[mask].copy()

In [ ]:
df_ikeda.describe()

In [ ]:
df_ikeda['score'].mean()

In [ ]:
B1_zeta_lambda = lambdify(sp.solve(equations.zeta_B1_equation, symbols.B_1)[0])
B2_d_lambda = lambdify(sp.solve(equations.d_B2_equation, symbols.B_2)[0])

def linearize(result):

    g=9.81
    rho=1000
    m = result.Volume*rho/(result.scale_factor**3)
    #result['B_1'] = B1_zeta_lambda(GM=result.gm, g=g, m=m, omega0=result.omega0, 
    #                                 zeta=result.zeta)
    #result['B_2'] = B2_d_lambda(GM=result.gm, g=g, m=m, omega0=result.omega0, d=result.d)
    
    factor=1.0  # Factor 
    phi_a = result.phi_start.abs()/factor  # Radians
    B_e_lambda=lambdify(sp.solve(equations.B_e_equation, symbols.B_e)[0])
    result['B_e'] = B_e_lambda(B_1=result['B_1'], B_2=result['B_2'], omega0=result.omega0, 
                                 phi_a=phi_a)
    
    return result

In [ ]:
#df_ikeda = linearize(df_ikeda)

In [ ]:
df_ikeda.describe()

In [ ]:
df_ikeda['score'].hist(bins=30)

In [ ]:
df_direct = database.load('rolldecay_quadratic_b',limit_score=0.7)
mask = (df_direct['ship_speed']==0)   ## Zero speed!
df_direct = df_direct.loc[mask].copy()
df_direct = linearize(df_direct)
mask = (df_direct['B_e'] < df_direct['B_e'].quantile(q=0.98))
df_direct=df_direct.loc[mask]

In [ ]:
df_direct['score'].hist(bins=30)

In [ ]:
df_direct['B_e'].hist(bins=30)

In [ ]:
df_ikeda['B_e'].hist(bins=30)

In [ ]:
df_compare = pd.merge(left=df_ikeda, right=df_direct, how='left', left_index=True, right_index=True,
                     suffixes=('_ikeda',''))
#df_compare.dropna(inplace=True,subset=['B_e','B_e_ikeda'])

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_e', y='B_e_ikeda', style='o', alpha=0.5, ax=ax)
xlim=ax.get_xlim()
ylim=ax.get_ylim()
ax.plot([0,xlim[1]],[0,xlim[1]],'r-')
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_aspect('equal', 'box')
ax.set_xlabel('$B_e$ (Model test)')
ax.set_ylabel('$B_e$ Ikeda')

In [ ]:
fig,ax=plt.subplots()
N=20
bins = np.linspace(df_compare['B_e_ikeda'].min(),df_compare['B_e_ikeda'].max(),N)
df_compare['B_e'].hist(bins=bins, ax=ax, label='linear')
df_compare['B_e_ikeda'].hist(bins=bins, ax=ax, label='linear', alpha=0.5)
